# Анализ сообщений пользователей чат-бота

В данном отчёте проводится анализ записанной в БД истории закодированных сообщений чат-бота, с целью получить ответы на следующие вопросы: 
- Определение общих метрик эффективности:
    * какова конверсия пользователя, написавшего в чат-бот, в создание заявки на оказание платной услуги?
    * какова конверсия из создания заказа в его завершение?
    * как распределён отток пользователей по шагам оформления заказа?
    * какова доля пользователей с запросом на оказание срочной услуги?
    * как в динамике менялись указанные выше показатели?
- Определение частных поведенческих метрик: 
    * какая доля пользователей просматривает информационный раздел о проекте?
    * какая доля пользователей использует опцию связи с администратором?
    * как много пользователей, отказывающихся от использования бота и сразу переключающихся на администратора?
    * меняется ли поведение пользователей в динамике (например, в зависимости от сезонности)?
- Определение метрик удобства использования бота: 
    * каковы самые частые ошибочные команды, которые вводят пользователи при использовании чат-бота и при каких сценариях использования они возникают?

Аналитическое заключение по выполненному анализу приведено в конце отчёта. 

***Технический комментарий***. Для визуализации указанных метрик на графиках используется библиотека **plotly**. 
Иногда возможны ситуации нарушения рендеринга построенных графиков - это связано с платформой или машиной, на которой просматривается данный отчёт. В этом случае рекомендуется попробовать обновить страницу, посмотреть позднее или скачать отчёт на локальную машину и посмотреть его там (предварительно потребуется установить указанные библиотеки). 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
'''
Демонстрация загрузки данных непосредственно через SQL-запрос к базе.
Ссылка для подключения к БД вида "postgresql:// ... " должна быть присвоена переменной db_connection 
в файле settings.
'''

from sqlalchemy import create_engine
import psycopg2
from settings import db_connection

con = create_engine(db_connection)

query_orders = '''SELECT * FROM orders'''
orders = pd.read_sql_query(query_orders, con, parse_dates = ['created_at', 'finished_at'])

query_messages = '''SELECT * FROM messages'''
messages = pd.read_sql_query(query_messages, con, parse_dates = ['created_at'])

In [5]:
'''
Для безопасности личных данных полученные описанным выше способом данные (с немного изменённым SQL-запросом)
экспортированы в csv и отправлены на Яндекс-Диск. Эти данные не обновляются, в отличие от тех, которые получены
способом выше. В дальнейшем анализируются именно они, в качестве демо-версии. 
Для анализа оперативных автообновляемых данных через прямой SQL-запрос к БД код необходимо немного изменить. 
'''

import requests
import urllib
import json

import warnings
warnings.filterwarnings('ignore')

folder_url = 'https://disk.yandex.ru/d/kCr2Plesa4vsOA'

messages_filename = 'messages.csv'

def create_df_from_url(source_url, file_url, sep = ','):
    yandex_download_api_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download'
    full_url = yandex_download_api_url + \
               '?public_key=' + urllib.parse.quote(source_url) + '&path=/' + urllib.parse.quote(file_url)
    r = requests.get(full_url) # запрос ссылки на скачивание
    h = json.loads(r.text)['href'] # 'парсинг' ссылки на скачивание
    df = pd.read_csv(h, sep, error_bad_lines=False, comment='#')
    return df

messages_all = create_df_from_url(folder_url, messages_filename)
messages_all = messages_all.drop(columns = 'Unnamed: 0')
messages_all['created_at'] = pd.to_datetime(messages_all['created_at'])
messages_all['month'] = messages_all['created_at'].dt.month
messages_all['group_id'] = np.where(messages_all['group_id'] == 101414135, "ECM",
                              (np.where(messages_all['group_id'] == 60444948, "Fin", 
                                       (np.where(messages_all['group_id'] == 50757966, "Stat", "Other")))))
messages_all = messages_all.rename(columns = {'group_id':'group'})

messages = messages_all[messages_all.msg_type == 'incoming']
messages.name = 'Incoming messages in 2021-2022'

# Анализ конверсии в начало и завершение заказа

**Вопрос данного раздела**: как много пользователей из тех, кто написал в бот, переходят к созданию заказа, и сколько из них его завершают?

In [8]:
all_users = \
    messages.groupby(['group', 'month', 'user_id'], as_index = False).count() \
            .groupby(['group', 'month'], as_index = False).count() \
            .rename(columns = {'user_id': 'users_count'}) \
            [['group', 'month', 'users_count']] \
            .query('month != 7 and month != 8') \
            .pivot_table('users_count', index='group', columns='month')

all_users = pd.concat({'all_users': all_users})

finished_orders = \
    messages_all[messages_all['msg_code'] == 'finish_order_msg'] \
        .groupby(['group', 'month'], as_index = False).agg({'user_id':'nunique'}) \
        .rename(columns = {'user_id':'orders_finished_by_unique'})

unique_users = all_users.melt(ignore_index = False) \
                       .reset_index().drop('level_0', axis=1) \
                       .rename(columns = {'value':'unique_users'})

order_msg = ['keyboard_off_cdo', 'keyboard_standard', 'keyboard_off_standard', 
             'keyboard_cdo', 'keyboard_off_project']

created_orders = \
    messages[np.isin(messages['msg_code'], order_msg)] \
        .query('month != 7') \
        .groupby(['group', 'month'], as_index = False).agg({'user_id':'nunique'}) \
        .rename(columns = {'user_id':'orders_created_by_unique'})

orders_table = unique_users \
                .merge(finished_orders, on = ['group', 'month']) \
                .merge(created_orders, on = ['group', 'month'])

orders_table['orders_finished_by_unique'] = \
    np.where(orders_table['orders_finished_by_unique'] > orders_table['orders_created_by_unique'],
             orders_table['orders_created_by_unique'], orders_table['orders_finished_by_unique'])

orders_table['CR_in_create_order'] = (orders_table['orders_created_by_unique']*100 / orders_table['unique_users']).round(1)
orders_table['CR_in_finish_order'] = (orders_table['orders_finished_by_unique']*100 / orders_table['orders_created_by_unique']).round(1)

fig = px.bar(
    data_frame=orders_table,
    x="month",
    y="CR_in_create_order",
    color="group", barmode="group",
    color_discrete_sequence=px.colors.qualitative.Dark2)

fig.update_xaxes(type='category', title_text='Месяц', 
                 categoryorder='array', categoryarray= [9, 10, 11, 12, 1, 2, 3, 4, 5, 6])
fig.update_yaxes(title_text='CR в создание заказа, %')
fig.update_layout(barmode='group', title_text = 'Конверсия в создание заказа по группам и периодам')
fig.show("svg")

fig2 = px.bar(
    data_frame=orders_table,
    x="month",
    y="CR_in_finish_order",
    color="group", barmode="group",
    color_discrete_sequence=px.colors.qualitative.Dark2)

fig2.update_xaxes(type='category', title_text='Месяц', 
                 categoryorder='array', categoryarray= [9, 10, 11, 12, 1, 2, 3, 4, 5, 6])
fig2.update_yaxes(title_text='CR в завершение заказа, %')
fig2.update_layout(barmode='group', title_text = 'Конверсия в завершение заказа по группам и периодам')
fig2.show()

min_CR_create = orders_table["CR_in_create_order"].min().round(1)
min_CR_finish = orders_table["CR_in_finish_order"].min().round(1)
min_CR_order = (min_CR_create * min_CR_finish / 100).round(1)

avg_CR_create = round(sum(orders_table["CR_in_create_order"]*orders_table['unique_users'])/sum(orders_table['unique_users']),1)
avg_CR_finish = round(sum(orders_table["CR_in_finish_order"]*orders_table['orders_created_by_unique'])/sum(orders_table['orders_created_by_unique']),1)
avg_CR_order = round(avg_CR_create * avg_CR_finish / 100,1)

print(f'Средний уровень конверсии в создание заказа: {avg_CR_create}%')
print(f'Средний уровень конверсии в завершение заказа: {avg_CR_finish}%')
print(f'Минимальная конверсия пользователя в заказчика: {min_CR_order}%, средняя конверсия: {avg_CR_order}%')

Средний уровень конверсии в создание заказа: 60.7%
Средний уровень конверсии в завершение заказа: 93.1%
Минимальная конверсия пользователя в заказчика: 26.7%, средняя конверсия: 56.5%


**Вывод.**

Наибольшая конверсия в создание заказа наблюдается в проекте *ECM* (55-75%), в двух других этот уровень меньше - от 40 до 60%. Уровень конверсии не имеет выраженной тенденции в течение рассматриваемого периода и колеблется вокруг в районе среднего уровня. 

Конверсия в завершение заказа существенно выше (от 70 до 100%), и по этому показателю нельзя выделить выраженных проектов-лидеров или аутсайдеров.

Из всех пользователей, обратившихся через чат-бот, около 56% завершили диалог оформлением заказа. Минимальная конверсия из обращения в заказ при наихудших уровнях из наблюдавшихся составила бы 27%. 

# Анализ прохождения пользователями шагов оформления заказа

**Вопросы данного раздела:**
1. На каких шагах происходит наибольший отток пользователей из чат-бота? 
2. Равномерно ли распределён отток пользователей по типу заказов?
3. В среднем какую часть общего пути оформления заказа проходит пользователь, прежде чем отказаться от дальнейшего взаимодействия с ботом?

Рассматриваем только тех пользователей, у которых не завершён ни один заказ. 

In [14]:
user_finish_orders = messages_all[messages_all['msg_code'] == 'finish_order_msg'].user_id.unique()
user_not_finish_orders = messages_all.query('user_id not in @user_finish_orders').user_id.unique()
msg_churn_users = messages_all[np.isin(messages_all['user_id'], user_not_finish_orders)]
msg_churn_users['msg_number'] = msg_churn_users \
                                    .sort_values(['created_at', 'user_id']) \
                                    .groupby(['user_id']) \
                                    .cumcount()+1

count_incoming_msg = []
max_num_codes = []
before_1_max_codes = []
before_2_max_codes = []

for id_user in user_not_finish_orders:
    count_msg = msg_churn_users[(msg_churn_users['user_id'] == id_user) & \
                                (msg_churn_users['msg_type'] == 'incoming')].shape[0]
    count_incoming_msg.append(count_msg)
    max_num_msg = msg_churn_users[msg_churn_users['user_id'] == id_user].msg_number.max()
    max_num_code = msg_churn_users[(msg_churn_users['user_id'] == id_user) & \
                                   (msg_churn_users['msg_number'] == max_num_msg)].msg_code
    max_num_codes.append(max_num_code)
    before_1_max_code = msg_churn_users[(msg_churn_users['user_id'] == id_user) & \
                                        (msg_churn_users['msg_number'] == max_num_msg-1)].msg_code
    before_1_max_codes.append(before_1_max_code)
    before_2_max_code = msg_churn_users[(msg_churn_users['user_id'] == id_user) & \
                                        (msg_churn_users['msg_number'] == max_num_msg-2)].msg_code
    before_2_max_codes.append(before_2_max_code)

count_incoming_msg = pd.Series(count_incoming_msg)
max_num_codes = [i.iloc[0] for i in max_num_codes]

for j in range(len(before_1_max_codes)):
    try:
        before_1_max_codes[j] = before_1_max_codes[j].iloc[0]
    except IndexError:
        before_1_max_codes[j] = ''
    
for j in range(len(before_2_max_codes)):
    try:
        before_2_max_codes[j] = before_2_max_codes[j].iloc[0]
    except IndexError:
        before_2_max_codes[j] = ''

print('Топ-10 кодов сообщений, после которых пользователь прекращает взаимодействие:')
display(pd.Series(max_num_codes).value_counts()[0:10])

print('Топ-10 кодов сообщений, предшествующих сообщению, после которого пользователь прекращает взаимодействие:')
display(pd.Series(before_1_max_codes).value_counts()[0:10])

print('Топ-10 кодов сообщений, дважды предшествующих сообщению, после которого пользователь прекращает взаимодействие:')
display(pd.Series(before_2_max_codes).value_counts()[0:10])

fig = px.histogram(count_incoming_msg[count_incoming_msg < 20], 
                   title = 'Отток пользователей в зависимости от количества отправленных сообщений',
                   histnorm='probability density')
fig.update_xaxes(title_text='Число сообщений, отправленных боту')
fig.update_yaxes(title_text='Отток пользователей, %')
fig.update_layout(showlegend=False)
fig.show("svg")

Топ-10 кодов сообщений, после которых пользователь прекращает взаимодействие:


answer_online          78
answer_no_emergency    58
answer_consulting      57
other                  52
unknown_command        45
1st_message_old        34
answer_offline         21
answer_cancel          18
standard_second_msg    15
answer_emergency       15
dtype: int64

Топ-10 кодов сообщений, предшествующих сообщению, после которого пользователь прекращает взаимодействие:


other                     117
keyboard_online            78
keyboard_no_emergency      58
keyboard_consulting        57
keyboard_menu              35
keyboard_offline           21
keyboard_cancel            18
outcoming_admin_menu_2     17
keyboard_emergency         16
keyboard_order             14
dtype: int64

Топ-10 кодов сообщений, дважды предшествующих сообщению, после которого пользователь прекращает взаимодействие:


                         114
answer_online             89
answer_order              43
other                     42
1st_message_old           30
unknown_command           22
answer_offline            19
answer_standard           17
incoming_admin_menu_2     17
answer_about              16
dtype: int64

**Результаты.**

Наибольший отток пользователей из чат-бота происходит при начале оформления онлайн-заказа (5,2%). В начале оформления пользователь видит общее число шагов, которое необходимо пройти для заполнения заявки. Это количество равно 6-7 и это может отпугивать часть аудитории. 

На втором месте по оттоку сообщение об отсутствии услуги срочной онлайн-помощи. Получая подобное сообщение, многие пользователи впоследствии не возвращаются (58 пользователей из 1496 - отток почти 4% только по этой причине).

На третьем месте - сообщение о наличии услуги бесплатной консультации. Отток после этого сообщения такой же, как и отток после сообщения об отсутствии срочных услуг. Возможно, стоит дополнить это сообщение информацией и кнопками для заказа платных услуг, чтобы пользователи могли пройти туда сразу же. 

Следующие два сообщения, после которых происходит отток - это сообщение пользователя в свободной форме, на которое, вероятно, не получен ответ и сообщение от бота об ошибке. Получив сообщение об ошибке, часть пользователей (3%) прекращает взаимодействие. После сообщения в свободной форме без получения ответа уходит ещё 3,5%.

Далее: первое сообщение в бот - около 2,3% пользователей завершило взаимодействие с ботом после его первого ответа. 
1,4% пользователей завершают коммуникацию после ответа при начале оформления услуги оффлайн (на этапе, когда необходимо выбрать подтип заказа). 

Те же выводы подтверждаются при анализе сообщений, предшествующих последнему. 
Всего пользователей, не совершивших ни одного заказа в боте - **около 40%**. По указанным критериям суммарный отток составляет **23,4%**. Остальные 16,6% прекращают взаимодействие, пройдя какое-то количество шагов оформления заказа.


В основном пользователи прекращают взаимодействие с ботом достаточно быстро - **число отправленных сообщений до прекращения взаимодействия по медиане равно 4**. 

In [15]:
def get_order_step(msg_code):
    order_steps = {1: ['keyboard_standard', 'keyboard_online'], 
                   2: ['standard_second_msg', 'off_st_second_msg', 'cdo_second_msg', 'off_cdo_second_msg', 'off_project_second_msg'], 
                   3: ['standard_number_tasks', 'off_st_doc', 'cdo_number_tasks', 'off_cdo_number_task', 'off_project_data'], 
                   4: ['standard_format', 'off_st_format', 'cdo_format', 'off_cdo_format', 'off_project_format'],
                   5: ['standard_time_limit', 'cdo_time_limit', 'off_st_time_date', 'off_cdo_time_limit', 'off_project_university'], 
                   6: ['standard_university', 'off_project_time_date', 'off_cdo_university', 'cdo_university'], 
                   7: ['off_st_time_date', 'standard_time_date', 'cdo_expectations', 'off_cdo_time_date', 'off_project_time_date']}
    
    for key, value in order_steps.items():
        if type(value) == list:
            for elem in value:
                if elem == msg_code:
                    return key
        elif value == msg_code:
            return key

msg_churn_users['order_step'] = msg_churn_users['msg_code'].apply(get_order_step)
churn_users_steps = msg_churn_users.groupby('user_id', as_index = False).agg({'order_step':'max'})
churn_users_steps = churn_users_steps[churn_users_steps['order_step'].notna()]
step_distr = churn_users_steps.order_step.value_counts()

fig = px.bar(pd.Series(step_distr*100/step_distr.sum()).round(2))
fig.update_xaxes(title_text='Число шагов в заказе')
fig.update_yaxes(title_text='Доля пользователей, %')
fig.update_layout(title_text='Число шагов в заказе, сделанных пользователями до прекращения диалога с ботом', 
                  showlegend=False)
fig.show("svg")

**Вывод.**

Подавляющее большинство пользователей из тех, кто не оформил заказ (80%) - прервали его оформление на первом шаге. Ещё 10% дошли до 2-го шага, а затем также прервали его. Таким образом, 90% пользователей, которые прошли 2 и более шага при оформлении заказа - завершили его оформление. 

Стоит рассмотреть информацию, которую получают пользователи на первых двух шагах и, возможно, упростить её / изменить тексты так, чтобы клиентов не пугало большое число шагов и они охотнее проходили по ним.

# Анализ конверсии раздела срочных обращений

**Вопрос данного раздела:** как много срочных обращений в чат-бот и отличается ли конверсия в эти обращения между группами и периодами?

In [9]:
emergency_msg_by_groups = \
    messages.groupby(['month', 'msg_code', 'group']) \
            .agg({'id':'nunique'}) \
            .pivot_table('id', index=['msg_code', 'group'], columns='month') \
            .fillna(0) \
            .astype('int') \
            .loc[['keyboard_emergency', 'keyboard_no_emergency']] \
            [[9, 10, 11, 12, 1, 2, 3, 4, 5, 6]]

emergency_total = emergency_msg_by_groups.loc['keyboard_emergency'] \
                      + emergency_msg_by_groups.loc['keyboard_no_emergency']

df_emergency = pd.concat([all_users, pd.concat({'emergency_total': emergency_total})])
df_emergency = df_emergency[[9, 10, 11, 12, 1, 2, 3, 4, 5, 6]]

CR_emergency_users = (df_emergency.loc['emergency_total']*100/ df_emergency.loc['all_users']).round(1)
CR_emergency_users = CR_emergency_users[[9, 10, 11, 12, 1, 2, 3, 4, 5, 6]]

print('Общее количество написавших в чат-бот и количество написавших по срочному обращению:')
display(df_emergency)

fig = go.Figure(data=[
    go.Bar(name=CR_emergency_users.index[0], x=CR_emergency_users.columns, y=CR_emergency_users.values[0]),
    go.Bar(name=CR_emergency_users.index[1], x=CR_emergency_users.columns, y=CR_emergency_users.values[1]),
    go.Bar(name=CR_emergency_users.index[2], x=CR_emergency_users.columns, y=CR_emergency_users.values[2]),
    ])
fig.update_xaxes(type='category')
fig.update_layout(barmode='group', title_text = 'График доли срочных обращений по группам и месяцам')
fig.show("svg")

Общее количество написавших в чат-бот и количество написавших по срочному обращению:


month                  9   10  11   12  1   2   3   4   5    6 
                group                                          
all_users       ECM    11  49  60  117  84  24  40  45  61   36
                Fin    14  40  48   60  68  13  26  26  35   56
                Stat   18  41  50   89  45  23  48  63  98  108
emergency_total ECM     1   7  13   31  10   1   7   4   8    6
                Fin     3  11  14   16  15   3   5   6  10   10
                Stat    5  16  17   24   6   7  17  11  19   28

**Вывод.**

Срочные обращения в чат-бот в относительном выражении меньше в проекте *ECM*, и примерно на одном уровне в остальных двух проектах. Наибольшая доля срочных обращений в 8 рассмотренных месяцах из 10 приходится на проект *Stat*. 

По динамике доли обращений видно, что выраженная сезонность (увеличение доли при приближении к сезонному периоду - началу зимы и лета) наблюдается лишь для группы *ECM*. 

# Анализ конверсии раздела **"О проекте"**

**Вопрос данного раздела**: как много пользователей просматривают раздел "О проекте" и его подразделы?

In [10]:
msg_unique_by_month = messages.groupby(['month', 'msg_code']) \
                              .agg({'user_id':'nunique'}) \
                              .pivot_table('user_id', index='msg_code', columns='month') \
                              .fillna(0) \
                              .astype('int')

msg_unique_by_month = msg_unique_by_month.drop(columns = 7)
msg_unique_by_month = msg_unique_by_month.drop(index = ['other', 'keyboard_beginning'])
msg_unique_by_month = msg_unique_by_month[[9, 10, 11, 12, 1, 2, 3, 4, 5, 6]]

msg_unique_about = msg_unique_by_month.loc[['keyboard_about', 'keyboard_conditions', 'keyboard_prices', 'keyboard_refs']]
msg_unique_about.loc['all_users'] = all_users.sum()

print('Количество уникальных пользователей по типу сообщения и месяцу в разделе "О проекте":')
display(msg_unique_about)

CR_menu_from_about_by_unique = msg_unique_about.loc['keyboard_about'] / msg_unique_about.loc['all_users']
CR_refs_from_about_by_unique = msg_unique_about.loc['keyboard_refs'] / msg_unique_about.loc['keyboard_about']
CR_cond_from_about_by_unique = msg_unique_about.loc['keyboard_conditions'] / msg_unique_about.loc['keyboard_about']
CR_price_from_about_by_unique = msg_unique_about.loc['keyboard_prices'] / msg_unique_about.loc['keyboard_about']


CR_to_about = pd.DataFrame([CR_menu_from_about_by_unique*100, CR_refs_from_about_by_unique*100,
                            CR_cond_from_about_by_unique*100, CR_price_from_about_by_unique*100]) \
                            .round(1).fillna(0)

CR_to_about.index = ['CR_to_about, %', 'CR_to_refs_from_about, %', 
                     'CR_to_conditions_from_about, %', 'CR_to_prices_from_about, %']

fig = px.bar(pd.DataFrame(CR_to_about.loc['CR_to_about, %']))
fig.update_xaxes(type='category', showgrid=True, ticks="outside", title_text='Месяц')
fig.update_yaxes(title_text='CR, %')
fig.update_layout(title_text='Конверсия в переход к разделу "О проекте" из меню:', 
                  showlegend=False)
fig.show("svg")

fig2 = go.Figure(data=[
    go.Bar(name=CR_to_about.index[1], x=CR_to_about.columns, y=CR_to_about.values[1]),
    go.Bar(name=CR_to_about.index[2], x=CR_to_about.columns, y=CR_to_about.values[2]),
    go.Bar(name=CR_to_about.index[3], x=CR_to_about.columns, y=CR_to_about.values[3]),
    ])
fig2.update_xaxes(type='category')
fig2.update_layout(barmode='group', 
                   title_text = 'Конверсия в переход к подразделам информации о проекте из общего раздела "О проекте"')
fig2.show("svg")

Количество уникальных пользователей по типу сообщения и месяцу в разделе "О проекте":


month,9,10,11,12,1,2,3,4,5,6
msg_code,,,,,,,,,,
keyboard_about,7,11,7,9,4,0,5,6,2,7
keyboard_conditions,0,1,1,1,2,0,0,1,1,1
keyboard_prices,5,7,4,6,4,0,2,4,2,3
keyboard_refs,0,1,1,1,0,0,0,0,0,0
all_users,43,130,158,266,197,60,114,134,194,200


**Вывод.** 

Раздел **"О проекте"** имеет низкую посещаемость среди всех пользователей. Он был популярен в первые два месяца запуска чат-бота, возможно, из интереса, но в дальнейшие месяцы этот эффект сошёл на нет. Доля пользователей, которые заходили в раздел **"О проекте"** в следующие месяцы не превышала 5%. 

При этом посетителей этого раздела в основном интересует цена услуги (в раздел **"Цены"** заходят от 40% до 100% всех посетителей раздела **"О проекте"**). Условиями работы и отзывами, напротив, интересуется меньшая часть клиентов. **"Отзывы"** из бота последние полгода вообще никто не открывал. Вероятно, этот раздел смотрят через интерфейс группы, а не чат-бота. 

# Анализ намерений выхода из чат-бота к диалогу с администратором

**Вопрос данного раздела:** много ли среди пользователей тех, кто переключает его в режим связи с администратором? Как много тех, кто делает это без попыток разобраться с интерфейсом бота?

In [11]:
msg_to_admin = msg_unique_by_month.loc[['incoming_admin_menu_1', 'incoming_admin_menu_2']]
msg_to_admin.loc['all_users'] = all_users.sum()

CR_admin_step1_to_menu = msg_to_admin.loc['incoming_admin_menu_1'] / msg_to_admin.loc['all_users']
CR_admin_step2_to_step1 = msg_to_admin.loc['incoming_admin_menu_2'] / msg_to_admin.loc['incoming_admin_menu_1']

CR_msg_to_admin = pd.DataFrame([CR_admin_step1_to_menu*100, CR_admin_step2_to_step1*100], 
                               index = ['CR_to_admin_step1_from_menu, %', 'CR_to_admin_step2_from_step1, %']) \
                    .round(1)

print('\nКоличество пользователей, переключающих чат-бота на администратора:')
display(msg_to_admin)

fig = go.Figure(data=[
    go.Bar(name=CR_msg_to_admin.index[0], x=CR_msg_to_admin.columns, y=CR_msg_to_admin.values[0]),
    go.Bar(name=CR_msg_to_admin.index[1], x=CR_msg_to_admin.columns, y=CR_msg_to_admin.values[1])
    ])
fig.update_xaxes(type='category')
fig.update_layout(barmode='group', 
                   title_text = 'Доля пользователей, переключающих чат-бота на администратора')
fig.show("svg")


Количество пользователей, переключающих чат-бота на администратора:


month,9,10,11,12,1,2,3,4,5,6
msg_code,,,,,,,,,,
incoming_admin_menu_1,4,13,12,9,14,6,21,16,26,42
incoming_admin_menu_2,2,10,6,3,8,4,11,10,14,19
all_users,43,130,158,266,197,60,114,134,194,200


**Вывод.**

На администратора переключается от 10 до 20% пользователей. Причём эта доля медленно увеличивается в течением времени, что является негативной тенденцией. Из них до следующего шага (окончательного перехода на чат с администратором) доходит от половины до 2/3 из тех, кто попытался воспользоваться этой опцией. Таким образом, предупреждающее сообщение уменьшает число обращений к администратору на **35-50%** от числа тех, кто имел такое намерение. С этой точки зрения оно весьма эффективно. 

Однако необходимо выяснить, много ли пользователей, которые включают режим обращения к администратору сразу, не пытаясь никак проходить через меню. 

In [12]:
users_msg_to_admin = messages[messages['msg_code'] == 'incoming_admin_menu_2'].user_id
user_story_msg_to_admin = messages[np.isin(messages['user_id'], users_msg_to_admin)]


msg_number = user_story_msg_to_admin.sort_values(['created_at', 'user_id']).groupby(['user_id']).cumcount()+1
user_story_ext_msg_to_admin = user_story_msg_to_admin
user_story_ext_msg_to_admin['message_number'] = msg_number

dist_of_admin_msg_number = user_story_ext_msg_to_admin.query('msg_code == "incoming_admin_menu_2"') \
                                                      .message_number \
                                                      .value_counts(normalize=True) \
                                                      .sort_index()

dist_msg_to_admin_short = dist_of_admin_msg_number.loc[0:10]
dist_msg_to_admin_short['more than 10'] = dist_of_admin_msg_number.loc[10:].sum()
df_dist_msg_to_admin = pd.DataFrame(round(dist_msg_to_admin_short*100,1)) \
                         .rename(columns = {'message_number':'% пользователей'})

fig = px.bar(df_dist_msg_to_admin)
fig.update_xaxes(type='category', showgrid=True, ticks="outside", title_text='Число сообщений')
fig.update_yaxes(title_text='% пользователей')
fig.update_layout(title_text='Распределение пользователей по порядковому номеру сообщения, вызывающего диалог с администратором:', 
                  showlegend=False)
fig.show()

**Вывод.**

Статистика показывает, что треть пользователей (из тех, кто включал режим связи с администратором) - включают этот режим сразу, даже не пытаясь пройти по пунктам меню чат-бота. Ещё 23% пользователей совершают не более 5 шагов в боте, после чего включают вызов администратора. Рассчитаем по максимальным месячным значениям конверсий долю пользователей, которые *не хотят* использовать чат-бота для обращений. 

In [223]:
max_refuse_prop = CR_msg_to_admin.iloc[0].max() * CR_msg_to_admin.iloc[1].max() * df_dist_msg_to_admin.iloc[0] / 1000000
max_refuse_prop = (max_refuse_prop[0] * 100).round(1)
print(f'Максимальная доля пользователей, отказывающихся от использования чат-бота: {max_refuse_prop}%')

Максимальная доля пользователей, отказывающихся от использования чат-бота: 4.8%


# Анализ ошибок пользователей при вводе команд

**Вопрос данного раздела:** на каких пунктах меню пользователи чаще всего вводят некорректные для бота команды. 

In [13]:
error_msg = ['unknown_command', 'incorrect_attachment_msg', 'incorrect_command', 'forward_alert_msg']
error_msg_ids = messages_all.query('msg_code in @error_msg')[['user_id', 'id']]

error_history_list = []
for msg_id in range(len(error_msg_ids)):
    user_id = error_msg_ids.iloc[msg_id][0]
    id = error_msg_ids.iloc[msg_id][1]
    user_msg = messages_all[messages_all['user_id'] == user_id].reset_index(drop = False)
    error_msg_index = user_msg[user_msg['id'] == id].index[0]
    try:
        error_msg_history = user_msg.iloc[[error_msg_index-2, error_msg_index-1, error_msg_index]]
    except IndexError:
        pass
    error_history_list.append(error_msg_history)
    
msg_code_2_before_error = []
msg_code_1_before_error = []
msg_code_error = []

for i in range(len(error_history_list)):
    msg_2_before_error = error_history_list[i].reset_index()['msg_code'][0]
    msg_1_before_error = error_history_list[i].reset_index()['msg_code'][1]
    msg_error = error_history_list[i].reset_index()['msg_code'][2]
    msg_code_2_before_error.append(msg_2_before_error)
    msg_code_1_before_error.append(msg_1_before_error)
    msg_code_error.append(msg_error)

print('Топ-10 кодов 2-го сообщения, предшествующего сообщению об ошибке:')
display(pd.Series(msg_code_2_before_error).value_counts()[0:10])

print('Топ-5 кодов 1-го сообщения, предшествующего сообщению об ошибке:')
display(pd.Series(msg_code_1_before_error).value_counts()[0:5])

print('Топ сообщений об ошибке:')
display(pd.Series(msg_code_error).value_counts())

Топ-10 кодов 2-го сообщения, предшествующего сообщению об ошибке:


other                     288
unknown_command            74
1st_message_old            32
answer_consulting          20
answer_online              19
outcoming_admin_menu_1     14
answer_later               11
answer_no_emergency        11
answer_offline             11
answer_order                7
dtype: int64

Топ-5 кодов 1-го сообщения, предшествующего сообщению об ошибке:


other                 394
no_attachments_msg    111
forward_alert_msg      21
keyboard_menu           5
keyboard_cancel         4
dtype: int64

Топ сообщений об ошибке:


unknown_command             498
forward_alert_msg            22
incorrect_attachment_msg     13
incorrect_command            11
dtype: int64

**Вывод.**

На 1-м месте по частоте ошибок пользователей находится ошибка неизвестной команды. В режиме включённого бота пользователи совершают ошибки в командах, вводя сообщение в свободной форме там, где такая возможность не предусмотрена, причём пытаются сделать это несколько раз, получая в ответ одно и то же сообщение об ошибке.
Остальные типы ошибок происходили относительно редко:
* 'forward_alert_msg' возникала, когда пользователи прикрепляли к сообщению перепост, а это допустимо лишь на этапе оформления заказа. Эта ошибка сопровождает ошибку 'unknown_command', следовательно, можно сделать вывод, что в 4% случаев пользователи пишут сообщения в свободной форме и прикрепляют перепост, вызывая эту ошибку. 
* 'incorrect_attachment' связана с прикреплением сообщений и возникает в аналогичных случаях. 
* 'incorrect_command' связана с вводом команды в момент оформления заказа, когда недоступны другие команды, кроме отмены заказа. Доля таких случаев незначительна. 

1-е сообщение, предшествующее сообщению об ошибке - обычно сообщение в свободной форме, которое и вызывает эту ошибку. А также сообщение о некорректном репосте или прикреплении. Эта информация не позволяет выяснить, на каком этапе это происходит. Для этого нужно посмотреть на 2-е предшествующее сообщение.

Первые два типа вторых предшествующих сообщений по частоте - это сообщения в свободной форме и сообщения об ошибке, которые указывают на то, что пользователи чаще всего совершают несколько попыток некорректных сообщений. Следующие за ними коды уже указывают на конкретные кейсы, когда пользователи совершают ошибки: 
* '1st_message_old' - при первом сообщении в чат-бот (при первом сообщении пользователь не должен получать сообщение об ошибке, эта проблема исправлена в более позднем обновлении, когда пользователь на любое первое сообщение, отличное от известной команды, получает приветственное сообщение);
* 'answer_consulting' - ошибка возникает, когда пользователь использует раздел, информирующий о бесплатной консультации и не прочитав эту информацию, вводит свой запрос в свободной форме;
* 'outcoming_admin_menu_1' - пользователи вводят сообщение для администратора после выбора соответствующего пункта меню, не прочитав предупреждающую информацию, которую бот выдаёт при выборе этого пункта;
* остальные пункты связаны с оформлением заказа, когда пользователи, выбрав соответствующий тип заказа, и не прочитав информацию, которую даёт бот, вводят сообщение в свободной форме, хотя для корректного прохождения сначала требуется уточнить ещё подтип заказа.

Количество ошибок пользователей на указанных шагах оценивается как **несущественное и не требующее доработки сценария**. 

# Резюме и рекомендации

На основе проведённого анализа были получены следующие результаты по основным метрикам. 

**По показателям конверсии написавшего в заказчика:**
* конверсия в создание заказа пользователем, написавшим боту, составляет 55-75% для проекта *ECM*, и 40-60% для проектов *Stat* и *Fin*. Таким образом, конверсия проекта *ECM* в среднем выше. Это может быть связано: 
    * с большим числом рекомендаций по типу "сарафанного радио" для этого проекта, по сравнению с другими;
    * более интенсивной конкуренцией в других двух проектах в силу их меньшей специфичности.
* конверсия в завершение заказа свыше 80% для всех проектов, существенных различий по этому критерию между ними нет;
* конверсия написавшего в покупателя варьируется от 30 до 60%, в зависимости от проекта и она существенно не меняется месяц к месяцу. 


**По показателям оттока пользователей:**
* сразу после начала коммуникации с ботом отток пользователей - 2,3% (от всех пользователей);
* отток после получения сообщения о наличии бесплатных консультаций через сообщения группы - 4%;
* отток после начала оформления услуги онлайн - 5,2%;
* отток после получения информации об отсутствии услуг срочного типа - 4%;
* отток после начала оформления услуги оффлайн - 1,4%;
* отток после получения сообщения об ошибке на всех этапах - 3%;
* отток после отсутствия ответа на сообщение в свободной форме (при обращении к администратору) - 3,5%;
* остальные сценарии - отток около 16%;
* медианный пользователь (по той части пользователей, которые не оформили ни одного заказа) отправляет не больше 4х сообщений боту до того, как прекратит взаимодействие. 


**Из тех пользователей, которые начали оформление заказа, но не дошли до конца:**
* 80% прекратили взаимодействие на 1-м шаге (вероятно, увидев количество шагов, которое необходимо пройти);
* 10% прекратили взаимодействие на 2-м шаге;
* Таким образом, подавляющее большинство пользователей, не доходящих до конца, проходят не больше 20% общего пути при заполнении заявки.


**По статистике срочных обращений:** 
* больше срочных обращений в относительном выражении приходится на осенне-зимний период, чем на весенне-летний (ежемесячная доля в осенне-зимний период на 5-10 процентных пунктов выше, чем в весенне-летний);
* внутри каждого из периодов существует тенденция к росту доли срочных обращений ближе к концу периода;
* проекты существенно отличаются по долей срочных обращений. Она наибольшая в проекте *Stat* и наименьшая в группе *ECM*. Вероятно, это связано с тем, что проект *Stat* в большей степени относится к категории масс-маркет.


**По посещениям раздела "О проекте":**
* данный раздел просматривает не более 4% пользователей;
* из тех, кто просматривает этот раздел, наибольшая доля посетителей (свыше 50% в большинство месяцев) смотрит раздел "Цены". Другие разделы просматривают в среднем не более 20% посетителей раздела "О проекте", т.е. не более 1% всех, написавших боту. 


**По поведению пользователей относительно вызова диалога с администратором:**
* от 10% до 20% пользователь используют опцию связи с администратором, но после предупреждающего сообщения их число сокращается до 5-15%;
* из тех пользователей, которые переключились в режим диалога с администратором, 30% сделали это сразу же после ответа бота, ещё 23% - не более, чем через 5 сообщений. Из оставшихся всего 30% пользователей сделали это, написав свыше 10 сообщений боту;
* сразу отказываются от использования бота при взаимодействии с сообществом не более 5% пользователей.

**По ошибкам пользователей при вводе команд:**
* чаще всего пользователи совершают ошибку, отправив боту сообщение в свободной форме там, где это не предусмотрено (на этапе до начала оформления заказа);
* Самые частые кейсы ошибок: 
    * ввод сообщения в свободной форме после выбора типа заказа, не выбрав подтип и не прочитав информацию этого раздела;
    * ввод запроса в свободной форме в разделе, информирующем о консультациях;
    * ввод сообщения после прохождения первого пункта меню обращения к администратору, не прочитав ответную информацию бота.  


## Рекомендации.

**Для сокращения оттока:**
* объяснить большое число необходимых шагов при прохождении заявки (возможно, ссылкой на отдельную статью);
* добавить в ответное сообщение о бесплатных консультациях кнопки / ссылки для перехода на форму заказа и соответствующие тексты;
* добавить ссылки для перехода на форму заказа из информационного раздела и перепроверить информацию на странице "Цены", сделав более информативной (т.к. она имеет высокую посещаемость); 


**Для увеличения числа пользователей бота:**
* проинформировать пользователей о случаях, в которых администратор может не отвечать (если такие имеются) и добавить информацию о возможности включения бота.


**Для сокращения ошибок пользователей:**
* проверить текст раздела консультаций, добавить предупреждение о том, что консультации проводятся не в чате.